In [1]:
# Montar o google drive no coolab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from transformers import BertTokenizer, RobertaTokenizer, BertTokenizerFast, BertModel, BertForSequenceClassification
import torch
import torch.nn.functional as F
import torch.nn as nn

from sklearn.metrics import classification_report
from sklearn import metrics

import pickle
import re
import os
from tqdm import tqdm
import ast
import random
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset


## Importando DataFrames

In [ ]:
df_train_use = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/train_clean_corpus.pkl')
df_test_use = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/test_clean_corpus.pkl')

In [ ]:
df_train_use.head()

,User_ID,Diagnosed_YN,Text,List_Timeline,List_CleanText,Qt_Messages
0,A_2002,1,"Ia pra praia cas miga, mas em Natal ta um chov...","[15-Dec-2017 10:07, 15-Dec-2017 15:37, 15-Dec-...","[Ia pra praia cas miga, mas em Natal ta um cho...",1199
1,A_645,1,ME APAIXONEI PELA MENINA QUE USA SALTO E TEM O...,"[09-Dec-2019 18:01, 10-Dec-2019 21:56, 11-Dec-...",[ME APAIXONEI PELA MENINA QUE USA SALTO E TEM ...,1579
2,A_735,1,"Decidi que apartir de agora, não vou mais dar ...","[28-Jun-2020 00:26, 28-Jun-2020 00:27, 28-Jun-...","[Decidi que apartir de agora, não vou mais dar...",1790
3,A_944,1,"hate pra cima do day6 na minha tml é unfollow,...","[08-Jun-2020 22:48, 10-Jun-2020 16:16, 10-Jun-...",[hate pra cima do day6 na minha tml é unfollow...,571
4,A_1198,1,"O saudade do meu branquelo, que não é meu AIND...","[26-Mar-2020 23:39, 26-Mar-2020 23:52, 27-Mar-...","[O saudade do meu branquelo, que não é meu AIN...",510


## Funções para métrica de avaliação f1-latency


Para cálculo da métrica de interesse f1-*latency* são necessárias algumas etapas representadas pelas funções abaixo.

In [6]:
def value_p(k):
    """Get the penalty value for the F latency measure.

    Parameters
    ----------
    k : int
        Median number of posts from the positive users.

    Returns
    -------
    penalty : float
        Penalty to use.
    """
    return -(np.log(1 / 3) / (k - 1))


def f_penalty(k, _p):
    """Get the penalty of the current user delay.

    Parameters
    ----------
    k : int
        Current user delay.
    _p : float
        Penalty.

    Returns
    -------
    f_penalty : float
        Penalty latency.
    """
    return -1 + (2 / (1 + np.exp((-_p) * (k - 1))))


def speed(y_pred, y_true, d, p):
    """Get speed for every user correctly classified as positive."""
    penalty_list = [
        f_penalty(k=d[i], _p=p)
        for i in range(len(y_pred))
        if y_pred[i] == 1 and y_true[i] == 1
    ]

    if len(penalty_list) != 0:
        return 1 - np.median(penalty_list)
    else:
        return 0.0


def f_latency(labels, true_labels, delays, penalty):
    """F latency metric.

    Metric proposed by Sadeque and others in [1]_.

    Parameters
    ----------
    labels : list of int
        Predicted label for each user.
    true_labels : list of int
        True label for each user.
    delays : list of int
        Decision delay for each user.
    penalty : float
        Penalty. Defines how quickly the penalty should increase.

    Returns
    -------
    f_latency_metric : float
        F latency measure.

    References
    ----------
    .. [1] `Sadeque, F., Xu, D., & Bethard, S. (2018, February). Measuring the
        latency of depression detection in social media. In Proceedings of the
        Eleventh ACM International Conference on Web Search and Data Mining
        (pp. 495-503).`_
    """
    f1_score = metrics.f1_score(y_pred=labels, y_true=true_labels, average="binary")
    speed_value = speed(y_pred=labels, y_true=true_labels, d=delays, p=penalty)

    return f1_score * speed_value

## Data prep

In [ ]:
df_train_use_BERT = df_train_use.copy()

## Fine-tunning BERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset


# Definindo o modelo BERT e tokenizer
# model_name = "neuralmind/bert-base-portuguese-cased"
model_name = 'pablocosta/bertabaporu-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Definindo o dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['Text']
        label = self.dataframe.iloc[idx]['Diagnosed_YN']
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        label = int(self.dataframe.iloc[idx]['Diagnosed_YN'])
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = CustomDataset(df_train_use_BERT, tokenizer)
# test_dataset = CustomDataset(test_df, tokenizer)

# Definindo o DataLoader
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Definindo os parâmetros de treinamento
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3

# Treinamento do modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Average training loss: {avg_train_loss}')

# Avaliando o modelo
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(train_loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculando a acurácia
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
print(f'Accuracy: {accuracy}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3:  66%|██████▌   | 784/1184 [1:03:19<33:30,  5.03s/it]

In [ ]:
print(classification_report(true_labels, predictions))

In [ ]:
# import pickle

# # Salvando o estado do modelo
torch.save(model.state_dict(), '/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/modelo_final.pth')

# # Salvando outros metadados importantes, se necessário
# # Por exemplo, salvar o tokenizer
with open('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/bert.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
# model_name = 'pablocosta/bertabaporu-base-uncased'
# # Para o modelo já treinado, quero carregar
# # Carregar o modelo a partir do arquivo .pkl, se necessário
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# # Carregando o estado do modelo
# model.load_state_dict(torch.load('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/modelo_final.pth'))

# # Carregando o tokenizer, se necessário
# with open('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/bert.pkl', 'rb') as f:
#     tokenizer = pickle.load(f)


## Política DMC - Momento de Decisão da Classificação
Os parâmetros usados nesta política foram de limiar = 0.5 e número mínimo de mensagens lidas de 20.

### Teste 1 - Utilizando a política como está descrita no artigo

In [ ]:
# df_test_use = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/test_clean_corpus.pkl')
df_test_use_BERT = df_test_use.copy()

# Vou utilizar a estratégia de cortar a timeline com a menor quantidade de mensagens
menor_timeline = df_test_use_BERT['Qt_Messages'].min()

def min_timeline(lista):
    return lista[- menor_timeline:]

df_test_use_BERT['List_CleanText_min'] = df_test_use_BERT['List_CleanText'].apply(min_timeline)
df_test_use_BERT.head()

In [ ]:
# parâmetros
delta = 0.5
n = 20

In [ ]:
# Definir o modelo da mesma maneira que você o definiu antes do treinamento
# model_name = 'pablocosta/bertabaporu-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Carregar o estado do modelo a partir do arquivo .pkl
# model.load_state_dict(torch.load('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/modelo_final.pth'), strict=False) # Versão coolab

# # Colocar o modelo em modo de avaliação
model.eval()

# # Criar lista para armazenar inputs tokenizados
tokenized_inputs = []

# print('tokenizando todos os dados..')
# # Tokenizar todos os dados
for text in tqdm(df_test_use_BERT['Text']):
    tokenized_input = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    tokenized_inputs.append(tokenized_input)

In [ ]:
# Salvamento dos dados tokenizados
# Definir o caminho para salvar os dados tokenizados
# save_path = '/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/testes_tokenizados.pkl'

# # Salvar os dados tokenizados
# with open(save_path, 'wb') as f:
#     pickle.dump(tokenized_inputs, f)

# Carregamento dos dados tokenizados
# Definir o caminho para carregar os dados tokenizados
# load_path = '/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/02. soft.bert/testes_tokenizados.pkl'

# Carregar os dados tokenizados
# with open(load_path, 'rb') as f:
#     tokenized_inputs = pickle.load(f)

In [ ]:
# Realizar a inferência
qtd_msgs_lidas = list(df_test_use_BERT['Qt_Messages'])
df_test_use_BERT['prediction'] = 0

print('aplicando a política..')
for i, tokenized_input in enumerate(tqdm(tokenized_inputs)):
    for j in range(len(df_test_use_BERT['List_CleanText_min'][i])):
        inputs_temp = {k: tokenized_input[k][:, j:j+10] for k in tokenized_input}

        # Realizar a inferência
        with torch.no_grad():
            outputs = model(**inputs_temp)

        # Interpretar os resultados
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        prob = torch.softmax(logits, dim=1)

        # Converter as previsões de volta para labels
        predicted_labels = [model.config.id2label[p.item()] for p in predictions]
        y_pred = [model.config.id2label[p.item()] for p in predictions]

        # Verificar se alguma previsão satisfaz os critérios
        max_prob, max_idx = torch.max(prob, dim=1)
        if (predictions[max_idx] == 1) and (max_prob >= delta):
            df_test_use_BERT.loc[i, 'prediction'] = 1
            qtd_msgs_lidas[i] = j + n
            break

        # if int(y_pred[0][-1]) == 1 and prob.max() >= delta:
        #     # df_test_use_BERT['prediction'][i] = 1
        #     df_test_use_BERT.loc[i, 'prediction'] = 1
        #     break
    # qtd_msgs_lidas.append(j + n)

df_test_use_BERT['qtd_msgs_lidas'] = qtd_msgs_lidas
# df_test_use_BERT.head()

In [3]:
# df_test_use_BERT.to_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_softBERT_MinTL.pkl')
df_test_use_BERT = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_softBERT_MinTL.pkl')

## Avaliação

In [ ]:
# df_test_use_BERT = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_softBERT_MinTL.pkl')
df_test_use_BERT.head()

In [4]:
y_train_teste_1 = df_test_use_BERT['Diagnosed_YN']
y_pred_teste_1 = df_test_use_BERT['prediction']
print(classification_report(y_train_teste_1, y_pred_teste_1))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93      3108
           1       0.00      0.00      0.00       444

    accuracy                           0.88      3552
   macro avg       0.44      0.50      0.47      3552
weighted avg       0.77      0.88      0.82      3552



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
f1_latency = f_latency(df_test_use_BERT['prediction'], df_test_use_BERT['Diagnosed_YN'], df_test_use_BERT['qtd_msgs_lidas'], 0.0078)
f1_latency

0.0

#### Tempo de antecipação

In [ ]:
df_test_use_BERT['List_Timeline'] = df_test_use_BERT['List_Timeline'].apply(eval)

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
from datetime import datetime

# Função para converter a lista de datas em objetos datetime
def converter_datas(lista_datas_str):
    # Remover os colchetes e espaços em branco
    # lista_datas_str = lista_datas_str.strip('[]').replace("'", "").split(', ')
    # Converter cada data para datetime
    return [pd.to_datetime(data_str, format='%d-%b-%Y %H:%M') for data_str in lista_datas_str]


# Função para calcular a diferença em dias entre duas datas
def calcular_diferenca_dias(data1, data2):
    delta = data1 - data2
    return delta.days

# Aplicando a conversão de datas ao DataFrame
df_test_use_BERT['List_Timeline'] = df_test_use_BERT['List_Timeline'].apply(converter_datas)

# Aplicando a função ao DataFrame
df_test_use_BERT['ultima_data_tweet_lido'] = df_test_use_BERT.apply(lambda row: row['List_Timeline'][row['qtd_msgs_lidas'] - 1], axis=1)
df_test_use_BERT['ultima_data_tweet_lista'] = df_test_use_BERT.apply(lambda row: row['List_Timeline'][-1], axis=1)
df_test_use_BERT['dias_antecipados'] = df_test_use_BERT.apply(lambda row: calcular_diferenca_dias(row['ultima_data_tweet_lista'], row['ultima_data_tweet_lido']), axis=1)

df_test_use_BERT.head()

,User_ID,Diagnosed_YN,Text,List_Timeline,List_CleanText,Qt_Messages,List_CleanText_min,prediction,qtd_msgs_lidas,ultima_data_tweet_lido,ultima_data_tweet_lista,dias_antecipados
0,A_1857,1,Esse Marcelo é um banana$END_OF_POST$Meu filho...,"[2014-02-18 23:28:00, 2014-02-18 23:28:00, 201...","[Esse Marcelo é um banana, Meu filho ninguém c...",2133,[Queria só que a sorte caminhasse ao meu lado ...,1,20,2014-02-20 22:17:00,2019-08-08 20:56:00,1994
1,A_1867,1,Fui tombada por esse site mas como sou fenix r...,"[2018-05-26 18:13:00, 2018-05-26 18:23:00, 201...",[Fui tombada por esse site mas como sou fenix ...,1081,[@ Só consegui reparar na minha antiga escola ...,1,20,2018-05-28 22:01:00,2019-07-29 10:21:00,426
2,A_414,1,Aquele momento gente em que tu não está com so...,"[2015-05-10 00:38:00, 2015-05-10 00:38:00, 201...",[Aquele momento gente em que tu não está com s...,1596,[@ A cria perfeita de dona Bianca no carnaval ...,1,20,2015-09-01 21:56:00,2021-01-18 22:14:00,1966
3,A_486,1,Tenho um xodó nesse user véi kkk Dá até vont...,"[2020-07-23 19:34:00, 2020-11-04 22:16:00, 202...",[Tenho um xodó nesse user véi kkk Dá até von...,957,"[@ Uhum, @ @ você segue a only jk, @ @ você se...",1,20,2020-11-05 12:01:00,2020-12-20 17:13:00,45
4,A_644,1,@ Fodasseeeeee$END_OF_POST$@ Luann com barba n...,"[2020-07-26 12:21:00, 2020-07-26 19:28:00, 202...","[@ Fodasseeeeee, @ Luann com barba não é luann...",103,"[Meu amigo tem um gosto peculiar, e o de vocês...",1,20,2020-09-22 11:36:00,2020-11-02 18:03:00,41


In [ ]:
dias_antecipacao_geral = df_test_use_BERT['dias_antecipados'].mean()
dias_antecipacao_pos = df_test_use_BERT[df_test_use_BERT['Diagnosed_YN']==1]['dias_antecipados'].mean()
print(f'Média de dias de antecipação de risco geral: {dias_antecipacao_geral}')
print(f'Média de dias de antecipação de risco da classe positiva: {dias_antecipacao_pos}')

Média de dias de antecipação de risco geral: 547.5478603603603
Média de dias de antecipação de risco da classe positiva: 507.64864864864865
